# Dataset Deduplication Tutorial


## Problem Statement

Exploratory data analysis (EDA) can be overwhelming. There are so many things to check.
Duplicates in your dataset, bad/corrupted images in the set, blurred or bright/dark images, the list goes on.

DataEval created a Duplicates class to assist you with your EDA so you can start training your models on high quality data.


### _When to use_

The Duplicates class should be used if you need to check for duplicates in your dataset.


### _What you will need_

1. A dataset to analyze
2. A python environment with the following packages installed:
   - `dataeval[torch]` or `dataeval[all]`
   - `torchvision`


## _Getting Started_

Let's import the required libraries needed to set up a minimal working example


In [ ]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval[torch]
except Exception:
    pass

In [ ]:
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms.v2 as v2

from dataeval.detectors.linters import Duplicates

## Loading in the data

Let's start by loading in torchvision's MNIST dataset, then we will examine it

The MNIST dataset contains 70,000 images - 60,000 in the train set and 10,000 in the test set.
For the purposes of this demonstration, we are just going to use the test set.


In [ ]:
# Load in the mnist dataset
to_tensor = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
testing_dataset = datasets.MNIST(root="./data/", train=False, transform=to_tensor, download=True)
test_data = np.array(testing_dataset.data, dtype=float)
labels = np.array(testing_dataset.targets)

Because the MNIST dataset does not contain any exact duplicates we are going to adjust the dataset to include some.


In [ ]:
# Creating some duplicates
print("Exact duplicates")
duplicates = {}
for i in [1, 2, 5, 9]:
    matching_indices = np.where(labels == i)[0]
    test_data[matching_indices[78]] = test_data[matching_indices[23]]
    print(f"\t{i} - ({matching_indices[23]}, {matching_indices[78]})")
    duplicates[i] = (matching_indices[23], matching_indices[78], matching_indices[2])

In [ ]:
print("Number of samples: ", len(test_data))

## Finding the Duplicates

Now we are asking our Duplicates class to find the needle in the haystack.
There are only 4 exact duplicates and then there are 3 shifted exact duplicates.


In [ ]:
# Initialize the Duplicates class
duplicator = Duplicates()

# Evaluate the data
results = duplicator.evaluate(test_data)

The results can be returned as a dictionary with exact and near as the keys. So we will extract those to view the results.


In [ ]:
for category, images in results.dict().items():
    print(f"{category} - {len(images)}")
    print(f"\t{images}")

If we recall from above, our exact duplicates were:

(180, 663), (212, 773), (219, 866), (249, 728)

Which exactly matches what the Duplicates class was able to find.

It also found several sets of images that are very closely related to each other, and since we are using hand written digits we would expect it to find some images that were nearly identical.
